# **Basic Data Analytics**

## Objectives

* In this notebook, we will do some preliminary statistical analysis, such as a correlation study, as well as various visualizations.

## Inputs

* The input for this is the cleaned data from the last notebook, namely `'game_data_clean.csv'`.

## Outputs

* At the end, we will have various plots displaying the statistical relationship between different features of our dataset.

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
home_dir = '/workspace/pp5-ml-dashboard'
csv_dir ='/workspace/pp5-ml-dashboard/outputs/datasets/clean/csv' 
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)



We now load our cleaned dataset as well as some of the packages that we will be using.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from ydata_profiling import ProfileReport
from src.utils import get_df

game_data = get_df('game_data_clean', csv_dir)
game_data.head()
#game_raw = get_df('game')

# Section 1: Correlation study and visualization
We hypothesize that many of the statistics will be correlated with each other. After dropping certain categorical features, lets look the correlation dataframe.

Note: add in playoff and regular season column to see if anything correlates there, and maybe leave year in, or add season in as an ordinal thing. maybe make team ordinal as well. drop day, maybe drop month as well.

In [ ]:
game_data.head()

We are going to modify the data frame before doing computing any exploratory data analysis. We are going to drop metadata  columns like `'game_id'`, `'team_id'`, as well as `'Day'` and `'Month'`. We will leave `'Year'` just in case something interesting shows up. We will also need to change the column `'wl_home'`.

In [ ]:
game_eda = game_data.drop(labels=['game_id','min','season_id', 'team_id_home', 'team_id_away', 'Day', 'Month'], axis=1)
game_eda['home_wins'] = game_eda.apply(lambda x: 1 if x['wl_home'] == 'W' else 0, axis=1)
game_eda.drop(labels=['wl_home','wl_away'], axis=1, inplace=True)

Let's look at a profile report produced by `ydata_profiling`.

In [ ]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=game_eda, minimal=True)
pandas_report.to_notebook_iframe()

In [ ]:

pearson_corr = game_eda.corr(method='pearson')['home_wins'].sort_values(key=abs, ascending=False)[1:]
print(pearson_corr[:11])
spearman_corr = game_eda.corr(method='spearman')['home_wins'].sort_values(key=abs, ascending=False)[1:]
print(spearman_corr[:11])

The following are the statistics with the least correlation with wins. It is a bit strange that there is any correlation between the home team winning and the year.

In [ ]:
print(pearson_corr[-5:])
print(spearman_corr[-5:])

This tells an interesting story. Aside from points, the statistic with the highest correlation to wins is the defensive rebounds of the opposing team. What about predictive power? Remember, this statistic is not symmetric.


This is a lot of data. So let's look at a heat map where we only consider correlation coefficients the top 9 correlation coefficients. Also, the `'plus_minus'` statistic is just the difference in points. So clearly, this will have the highest correlation with who wins (why is it not a correlation coefficient of 1?). The Spearman and Pearson correlation coefficients indicate the same features for further study. We may also remove the `'pts'` statistics.

In [ ]:
vars_to_study = ['ast_home','ast_away', 'dreb_away', 'dreb_home', 'fgm_away', 'fgm_home', 'pts_away', 'pts_home']
top_corr = game_data.filter(list(vars_to_study)+['home_wins'])
top_corr.head()

---

These are all numerical fields. So we will plot the distribution of the values and use hue to distinguish between wins and losses. Notice that there is a certain symmetry in the distributions. If the role of home and away are swapped we could also swap win and loss. The symmetry can also be thought of as reflecting across the y-axis. This shows a genuine symmetry in the data since each game is only listed once. The presence of the symmetry is almost reassuring and is very intuitive.

In [ ]:
for var in vars_to_study:
    plt.figure(figsize=(8, 5))
    sns.histplot(data=game_data_for_corr, x=var, hue='home_wins', kde=True, element="step")
    plt.title(f"{var}", fontsize=20, y=1.05)
    plt.show()

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
